# 범주형 데이터 전처리

## 인코딩

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager,rc

if platform.system()=='Darwin':
	rc('font',family='AppleGothic')
elif platform.system()=='Windows':
	font_name=font_manager.FontProperties(fname='c:/windows/Fonts/malgun.ttf').get_name()
	rc('font',family=font_name)

	
# 그래프에 음수를 사용하기 위한 설정
plt.rcParams['axes.unicode_minus']=False

In [3]:
auto_mpg=pd.read_csv('./data 4/auto-mpg.csv',header=None)
auto_mpg.columns=['mpg','cylinders','displacement','horsepower','weight','acceleration','model year','origin','name']

# horsepower 열의 자료형을 실수로 변경
# ?를 None으로 치환하고 제거한 후 자료형 변경
auto_mpg['horsepower'].replace('?',np.nan,inplace=True)
auto_mpg.dropna(subset=['horsepower'],axis=0,inplace=True)
auto_mpg['horsepower']=auto_mpg['horsepower'].astype("float")
auto_mpg


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625.0,18.6,82,1,ford ranger


In [4]:
# 원 핫 인코딩

# horsepower 특성을 범주형으로 추가 - 3개의 영역으로 구분

# 3개의 구간으로 구분해서 개수와 경계값을 리턴 받아서 저장
count,bin_dividers=np.histogram(auto_mpg['horsepower'],bins=3)

# 범주형의 형태로 생성

auto_mpg['hp_bin']=pd.cut(x=auto_mpg['horsepower'],bins=bin_dividers,labels=['저출력','보통출력','고출력'],include_lowest=True)

# one-hot encoding
horsepower_dummies=pd.get_dummies(auto_mpg['hp_bin'])
horsepower_dummies

,저출력,보통출력,고출력
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
393,1,0,0
394,1,0,0
395,1,0,0
396,1,0,0


In [5]:
from sklearn.preprocessing import LabelBinarizer

one_hot=LabelBinarizer()
print(one_hot.fit_transform(auto_mpg[['hp_bin']]))

# 이름을 확인
print(one_hot.classes_)

[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]
['고출력' '보통출력' '저출력']


In [6]:
# 2개 이상의 특성을 가지고 원 핫 인코딩
# 2개 이상의 1이 등장할 수 있다. 
from sklearn.preprocessing import MultiLabelBinarizer

multi_feature=[('java','python'),('C++','C#'),('C++','kotilin'),('objective-c','python'),('GO','R')]
one_hot=MultiLabelBinarizer()
print(one_hot.fit_transform(multi_feature))

print(one_hot.classes_)

[[0 0 0 0 1 0 0 1]
 [1 1 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 1]
 [0 0 1 1 0 0 0 0]]
['C#' 'C++' 'GO' 'R' 'java' 'kotilin' 'objective-c' 'python']


In [7]:
# 순서가 의미를 갖는 경우 - replace 함수 이용
df=pd.DataFrame({"Score":['저조','보통','보통','저조','우수','매우우수']})
scale_mapper={"저조":1,"보통":2,'우수':3,'매우우수':4}
df['encoder']=df['Score'].replace(scale_mapper)
df

,Score,encoder
0,저조,1
1,보통,2
2,보통,2
3,저조,1
4,우수,3
5,매우우수,4


In [8]:
# 2개 이상의 인코딩
from sklearn.preprocessing import OrdinalEncoder

features=np.array([['low',10],['normal',20],['high',15]])

ordinal_encoder=OrdinalEncoder()
print(ordinal_encoder.fit_transform(features))
print(ordinal_encoder.categories_)

[[1. 0.]
 [2. 2.]
 [0. 1.]]
[array(['high', 'low', 'normal'], dtype='<U21'), array(['10', '15', '20'], dtype='<U21')]


## 결측치 대체

In [9]:
# 분류 모델을 이용한 결측값 대체
from sklearn.neighbors import KNeighborsClassifier

# 훈련 데이터 생성
X=np.array([[0,2.10,1],[1,1.18,1.34],[0,1.21,2],[1,2,1.34]])

# 예측에 사용할 데이터
X_with_nan=np.array([[np.nan,0.87,0.31],[np.nan,-0.67,0.22]])

clf=KNeighborsClassifier(3,weights='distance')
# 첫번째 데이터를 label로 하고 나머지 데이터를 feature 로 설정해서 훈련한다. 
trained_model=clf.fit(X[:,1:],X[:,0])
# 예측 
imputed_values=trained_model.predict(X_with_nan[:,1:])
print(imputed_values)

# 예측 데이터와 원본 데이터를 합친다. 
X_with_imputed=np.hstack((imputed_values.reshape(-1,1),X_with_nan[:,1:]))
print(X_with_imputed)

# 결측치를 대체한 데이터와 훈련에 사용한 데이터 합치기/ 
result=np.vstack((X_with_imputed,X))
result

[1. 0.]
[[ 1.    0.87  0.31]
 [ 0.   -0.67  0.22]]


array([[ 1.  ,  0.87,  0.31],
       [ 0.  , -0.67,  0.22],
       [ 0.  ,  2.1 ,  1.  ],
       [ 1.  ,  1.18,  1.34],
       [ 0.  ,  1.21,  2.  ],
       [ 1.  ,  2.  ,  1.34]])

In [10]:
# 가장 많이 나오는 데이터로 대체
from sklearn.impute import SimpleImputer

X_complete=np.vstack((X_with_nan,X))
print(X_complete)
imputer=SimpleImputer(strategy='most_frequent') # 최빈값 대체.
imputer.fit_transform(X_complete)

[[  nan  0.87  0.31]
 [  nan -0.67  0.22]
 [ 0.    2.1   1.  ]
 [ 1.    1.18  1.34]
 [ 0.    1.21  2.  ]
 [ 1.    2.    1.34]]


array([[ 0.  ,  0.87,  0.31],
       [ 0.  , -0.67,  0.22],
       [ 0.  ,  2.1 ,  1.  ],
       [ 1.  ,  1.18,  1.34],
       [ 0.  ,  1.21,  2.  ],
       [ 1.  ,  2.  ,  1.34]])

# 텍스트 데이터 전처리

### 정규 표현식

In [11]:
import re

# 매칭여부를 확인
match=re.match('[0-9]','1234')
# 패턴에 일치하는 데이터가 있으면 match 객체를 리턴하고 없다면 none 리턴
print(match)

match=re.match('[0-9]','abc')
print(match)# 매치 데이터가 없어 none 출력

<re.Match object; span=(0, 1), match='1'>
None


In [12]:
match=re.match('[0-9]',' 1234') # 공백이 있다면 매치를 못한다. 
match2=re.match('\s[0-9]',' 1234') # 공백이 있다면 매치를 못한다. -> 공백 제거 추가.  
print(match2)

<re.Match object; span=(0, 2), match=' 1'>


In [13]:
string="@안녕하세요 반갑습니다^^ 123 의미없는 숫자 .....!!!!"

# 숫자 데이터 제거
p=re.compile('[0-9]+')
result=p.sub('',string)
print(result)

p=re.compile("\W+")
result=p.sub(' ',result)
print(result)

@안녕하세요 반갑습니다^^  의미없는 숫자 .....!!!!
 안녕하세요 반갑습니다 의미없는 숫자 


In [14]:
import unicodedata
import sys

text_data=['안녕하세요 반갑습니다.','My job is Programmer','C&C++ ,C#, Python']

# 구두점 딕셔너리를 생성
punctuation=dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))
result=[string.translate(punctuation) for string in text_data]
result

['안녕하세요 반갑습니다', 'My job is Programmer', 'CC++ C Python']

### 텍스트 토큰화

In [15]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/kimjimin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kimjimin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.tokenize import word_tokenize # 단어단위 토큰화
from nltk.tokenize import sent_tokenize # 문장단위 토큰화

string="오펜하이머는 얼마나 좋았을까. 남들 못가는 하버드 대학교도 가고."
print(word_tokenize(string))

print(sent_tokenize(string))

['오펜하이머는', '얼마나', '좋았을까', '.', '남들', '못가는', '하버드', '대학교도', '가고', '.']
['오펜하이머는 얼마나 좋았을까.', '남들 못가는 하버드 대학교도 가고.']


In [17]:
from nltk.corpus import stopwords
# 불용어 제거
word_Korean=['1월','2월','3월','4월']
stopword=['2월','5월']
# i for i in word_Korean if i not in stopwords 는 작업을 수행해서 generator 를 생성
# generator 는 iterator 로 접근할 수 있는 객체
result=[i for i in word_Korean if i not in stopword]
print(result)

# 영문은 nltk 에서 기본적인 불용어 사전을 제공. 
word_eng=['chief','the','an','and','president','kenedy','move']
result=[w for w in word_eng if not w in stopwords.words('english')]
print(result)

['1월', '3월', '4월']
['chief', 'president', 'kenedy', 'move']


In [18]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
result=[w for w in word_eng if not w in ENGLISH_STOP_WORDS]
print(result)

['chief', 'president', 'kenedy']


### 어간 추출

In [19]:
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

from nltk.tokenize import word_tokenize


string="All pythoners have pythoned poorly at least once"

words=word_tokenize(string)

ps_stemmer=PorterStemmer()
print(words)


result=[ps_stemmer.stem(i) for i in words]
print(result)

LC_stemmer=LancasterStemmer()
result=[LC_stemmer.stem(i) for i in words]
print(result)

['All', 'pythoners', 'have', 'pythoned', 'poorly', 'at', 'least', 'once']
['all', 'python', 'have', 'python', 'poorli', 'at', 'least', 'onc']
['al', 'python', 'hav', 'python', 'poor', 'at', 'least', 'ont']


### 형태소 분석

In [20]:
import nltk
nltk.download('averaged_perceptron_tagger')

from nltk import pos_tag
from nltk import word_tokenize

tokens=word_tokenize("Hi!, my name is jason Kim. I'm Interested in NLP")
tags_en=pos_tag(tokens)
print(tags_en) # 단어와 품사의 튜플 리스트로 출력. 
# 품사 ( NNP- 고유명사 , NN- 명사 , RB- 부사, VBD- 동사 , VBG- 동사 , 동명사,현재분사, JJ- 형용사)\
print([word for word,tag in tags_en if tag in ['NN','NNP']])

[('Hi', 'NN'), ('!', '.'), (',', ','), ('my', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('jason', 'JJ'), ('Kim', 'NNP'), ('.', '.'), ('I', 'PRP'), ("'m", 'VBP'), ('Interested', 'JJ'), ('in', 'IN'), ('NLP', 'NNP')]
['Hi', 'name', 'Kim', 'NLP']


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kimjimin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### 한국어 분석
* 성능은 kkma 가 우수하다고 알려져 있지만 메모리 사용량이 많고 속도가 조금 느리다. 

In [21]:
from konlpy.tag import Kkma
from konlpy.tag import Hannanum

In [22]:
text="김한경은 1998년 6월 29일 생으로 현재 당산역 앞 삼성 레미안 아파트에서 거주하고 있다. "
kkma=Kkma()
# 문장 분석 
print(kkma.sentences(text))
# 단어 분석
print(kkma.nouns(text))
# 형태소 분석 
print(kkma.pos(text))

['김한 경은 1998년 6월 29일 생으로 현재 당산 역 앞 삼성 레 미안 아파트에서 거주하고 있다.']
['김', '경', '1998', '1998년', '년', '6', '6월', '월', '29', '29일', '일', '현재', '당산', '당산역', '역', '앞', '삼성', '레', '레미안', '미안', '아파트', '거주']
[('김', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('경', 'NNG'), ('은', 'JX'), ('1998', 'NR'), ('년', 'NNM'), ('6', 'NR'), ('월', 'NNM'), ('29', 'NR'), ('일', 'NNM'), ('생으로', 'MAG'), ('현재', 'NNG'), ('당산', 'NNP'), ('역', 'NNG'), ('앞', 'NNG'), ('삼성', 'NNG'), ('레', 'NNG'), ('미안', 'NNG'), ('아파트', 'NNG'), ('에서', 'JKM'), ('거주', 'NNG'), ('하', 'XSV'), ('고', 'ECE'), ('있', 'VXV'), ('다', 'EFN'), ('.', 'SF')]


In [23]:
han=Hannanum()
# 형태소 분석
print(han.pos(text))
# 단어 분석
print(han.nouns(text))

[('김한경', 'N'), ('은', 'J'), ('1998년', 'N'), ('6월', 'N'), ('29일', 'N'), ('생', 'N'), ('으로', 'J'), ('현재', 'M'), ('당산역', 'N'), ('앞', 'N'), ('삼성', 'N'), ('레미안', 'N'), ('아파트', 'N'), ('에서', 'J'), ('거주', 'N'), ('하고', 'J'), ('있', 'P'), ('다', 'E'), ('.', 'S')]
['김한경', '1998년', '6월', '29일', '생', '당산역', '앞', '삼성', '레미안', '아파트', '거주']


### BOW (bag of words - 단어의 개수)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

text_data=np.array(['I love Korea korea','I love real Madrid','I hate Barcelona','Gemany minjae kim'])
co=CountVectorizer()
bag_of_words=co.fit_transform(text_data)

print(co.get_feature_names_out())
# 희소행렬의 형태로 출력됨. 
print(bag_of_words)
# 밀집 행렬의 형태로 변경
print(bag_of_words.toarray())

['barcelona' 'gemany' 'hate' 'kim' 'korea' 'love' 'madrid' 'minjae' 'real']
  (0, 5)	1
  (0, 4)	2
  (1, 5)	1
  (1, 8)	1
  (1, 6)	1
  (2, 2)	1
  (2, 0)	1
  (3, 1)	1
  (3, 7)	1
  (3, 3)	1
[[0 0 0 0 2 1 0 0 0]
 [0 0 0 0 0 1 1 0 1]
 [1 0 1 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 1 0]]


### td-idf ( 단어의 가중치 )

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_data=np.array(['I real Madrid love','I love real Madrid','I hate Barcelona','Gemany minjae kim'])
tf=TfidfVectorizer()
feature_matrix=tf.fit_transform(text_data)

print(tf.get_feature_names_out())
# 희소행렬의 형태로 출력됨. 
print(feature_matrix)
# 위치에 해당하는 단어가 나온다.
print(tf.vocabulary_)
# 밀집 행렬의 형태로 변경
print(feature_matrix.toarray())

import pandas as pd
print(pd.DataFrame(feature_matrix.toarray()))

['barcelona' 'gemany' 'hate' 'kim' 'love' 'madrid' 'minjae' 'real']
  (0, 4)	0.5773502691896257
  (0, 5)	0.5773502691896257
  (0, 7)	0.5773502691896257
  (1, 4)	0.5773502691896257
  (1, 5)	0.5773502691896257
  (1, 7)	0.5773502691896257
  (2, 0)	0.7071067811865476
  (2, 2)	0.7071067811865476
  (3, 3)	0.5773502691896257
  (3, 6)	0.5773502691896257
  (3, 1)	0.5773502691896257
{'real': 7, 'madrid': 5, 'love': 4, 'hate': 2, 'barcelona': 0, 'gemany': 1, 'minjae': 6, 'kim': 3}
[[0.         0.         0.         0.         0.57735027 0.57735027
  0.         0.57735027]
 [0.         0.         0.         0.         0.57735027 0.57735027
  0.         0.57735027]
 [0.70710678 0.         0.70710678 0.         0.         0.
  0.         0.        ]
 [0.         0.57735027 0.         0.57735027 0.         0.
  0.57735027 0.        ]]
          0        1         2        3        4        5        6        7
0  0.000000  0.00000  0.000000  0.00000  0.57735  0.57735  0.00000  0.57735
1  0.000000  0.0